In [70]:
import pandas as pd
import re

# Load data from Excel
file = 'p24.xlsx'
p = pd.read_excel(file, sheet_name='p24')
p['kwota'] = p['kwota'].astype(int)
p['data'] = pd.to_datetime(p['data'])
p['tytul'] = p['tytul'].astype(str)

b = pd.read_excel(file, sheet_name='BC')
b['kwota'] = (b['kwota'] * 100).round().astype(int)
b['data'] = pd.to_datetime(b['data'])
b['tytul'] = b['tytul'].astype(str)

# Define function to extract reservation numbers
def find_res(text):
    res = re.findall(r'(\D|\b)(\d{7})(\D|\b)', text)
    return res[0][1] if res else None

# Add 'res' columns to both DataFrames
p['res'] = p['tytul'].apply(find_res)
b['res'] = b['tytul'].apply(find_res)

# Merge DataFrames on 'res' and 'kwota'
merged = p.merge(b, on=['res', 'kwota'], suffixes=('_p', '_b'), how='outer')

# save merged to Excel
merged.to_excel('merged.xlsx', index=False)

# Separate rows into different categories
matches_same_month = merged[
    (merged['data_p'].dt.month == merged['data_b'].dt.month) & merged['res'].notna()
]
matches_diff_month = merged[
    (merged['data_p'].dt.month != merged['data_b'].dt.month) & merged['res'].notna()
]
unmatched = merged[
    merged['res'].isna() | merged[['data_p', 'data_b']].isnull().any(axis=1)
]

# Save results to Excel
with pd.ExcelWriter('output.xlsx') as writer:
    matches_same_month.to_excel(writer, sheet_name='matches_same_month', index=False)
    matches_diff_month.to_excel(writer, sheet_name='matches_diff_month', index=False)
    unmatched.to_excel(writer, sheet_name='unmatched', index=False)
